In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from transformers import BertModel, BertTokenizer
import torch

def load_tsv_data(file_path):
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                review = parts[0]
                features = parts[1:]
                for feature in features:
                    feature_parts = feature.split()
                    if len(feature_parts) >= 4:
                        aspect_term_indices = feature_parts[0].split(',')
                        sentiment = feature_parts[2]
                        sentiment_term_indices = feature_parts[3].split(',')
                        
                        if aspect_term_indices[0] == '-1':
                            aspect_term = 'NULL'
                        else:
                            start, end = int(aspect_term_indices[0]), int(aspect_term_indices[1])
                            aspect_term = ' '.join(review.split()[start:end])
                        
                        if sentiment_term_indices[0] == '-1':
                            sentiment_term = 'NULL'
                        else:
                            start, end = int(sentiment_term_indices[0]), int(sentiment_term_indices[1])
                            sentiment_term = ' '.join(review.split()[start:end])
                        
                        data.append({
                            'review': review,
                            'aspect_term': aspect_term,
                            'sentiment_term': sentiment_term,
                            'sentiment': sentiment
                        })
            else:
                print(f"Skipping malformed line: {line.strip()}")
    
    return pd.DataFrame(data)

def train_and_tune_model(X_train, y_train, X_dev, y_dev):
    try:
        # define the parameter grid
        param_grid = {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto', 0.1, 1]
        }
        
        # perform grid search with cross-validation
        grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        
        # get the best model
        best_model = grid_search.best_estimator_
        
        # evaluate the best model on the dev
        dev_accuracy = best_model.score(X_dev, y_dev)
        print(f"Best model parameters: {grid_search.best_params_}")
        print(f"Development set accuracy: {dev_accuracy}")
        
        return best_model
    except Exception as e:
        print(f"Error during model training and tuning: {e}")
        return None

def evaluate_model(model, X_test, y_test):
    try:
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        return accuracy, report
    except Exception as e:
        print(f"Error during model evaluation: {e}")
        return None, None

review

In [2]:
def extract_features1(reviews, model, tokenizer):
    features = []
    for text in reviews:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        features.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(features)

def baseline_experiment1(train_data, dev_data, test_data, model, tokenizer):
    
    print("\nExtracting features...")
    X_train = extract_features1(train_data['review'], model, tokenizer)
    X_dev = extract_features1(dev_data['review'], model, tokenizer)
    X_test = extract_features1(test_data['review'], model, tokenizer)
    
    print("Encoding labels...")
    le = LabelEncoder()
    le.fit(train_data['sentiment'].tolist() + dev_data['sentiment'].tolist() + test_data['sentiment'].tolist())
    y_train = le.transform(train_data['sentiment'])
    y_dev = le.transform(dev_data['sentiment'])
    y_test = le.transform(test_data['sentiment'])
    
    print("Training and tuning model...")
    best_classifier = train_and_tune_model(X_train, y_train, X_dev, y_dev)
    
    print("Evaluating model...")
    accuracy, report = evaluate_model(best_classifier, X_test, y_test)
    
    return best_classifier, le, accuracy, report


if __name__ == "__main__":
    bert_model = BertModel.from_pretrained(r'D:\AIProject\Bert\model')
    bert_tokenizer = BertTokenizer.from_pretrained(r'D:\AIProject\Bert\tokenizer')

    # restuarant
    rest_train_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_train.tsv')
    rest_test_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_test.tsv')
    rest_dev_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_dev.tsv')
    
    print("\nRunning baseline experiment...")
    baseline_classifier, baseline_le, baseline_accuracy1, baseline_report1 = baseline_experiment1(rest_train_data, rest_dev_data, rest_test_data, bert_model, bert_tokenizer)
    
    print("\nBaseline Experiment Results:")
    print(f"Accuracy: {baseline_accuracy1}")
    print("Classification Report:")
    print(baseline_report1)

    print("\nBaseline Experiments completed.")


Running baseline experiment...

Extracting features...
Encoding labels...
Training and tuning model...
Best model parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Development set accuracy: 0.8659003831417624
Evaluating model...

Baseline Experiment Results:
Accuracy: 0.8417030567685589
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.77      0.72       205
           1       0.00      0.00      0.00        44
           2       0.90      0.92      0.91       667

    accuracy                           0.84       916
   macro avg       0.52      0.56      0.54       916
weighted avg       0.81      0.84      0.82       916


Baseline Experiments completed.


c:\Program Files\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


reviews, aspect_terms

In [3]:
def extract_features2(reviews, aspect_terms, model, tokenizer):
    features = []
    for review, aspect_term in zip(reviews, aspect_terms):
        inputs = tokenizer(review, aspect_term, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        features.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(features)

def baseline_experiment2(train_data, dev_data, test_data, model, tokenizer):
    
    print("\nExtracting features...")
    X_train = extract_features2(train_data['review'], train_data['aspect_term'], model, tokenizer)
    X_dev = extract_features2(dev_data['review'], dev_data['aspect_term'], model, tokenizer)
    X_test = extract_features2(test_data['review'], test_data['aspect_term'], model, tokenizer)
    
    print("Encoding labels...")
    le = LabelEncoder()
    le.fit(train_data['sentiment'].tolist() + dev_data['sentiment'].tolist() + test_data['sentiment'].tolist())
    y_train = le.transform(train_data['sentiment'])
    y_dev = le.transform(dev_data['sentiment'])
    y_test = le.transform(test_data['sentiment'])
    
    print("Training and tuning model...")
    best_classifier = train_and_tune_model(X_train, y_train, X_dev, y_dev)
    
    print("Evaluating model...")
    accuracy, report = evaluate_model(best_classifier, X_test, y_test)
    
    return best_classifier, le, accuracy, report

if __name__ == "__main__":
    bert_model = BertModel.from_pretrained(r'D:\AIProject\Bert\model')
    bert_tokenizer = BertTokenizer.from_pretrained(r'D:\AIProject\Bert\tokenizer')

    # restuarant
    rest_train_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_train.tsv')
    rest_test_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_test.tsv')
    rest_dev_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_dev.tsv')
    
    print("\nRunning baseline experiment...")
    baseline_classifier, baseline_le, baseline_accuracy2, baseline_report2 = baseline_experiment2(rest_train_data, rest_dev_data, rest_test_data, bert_model, bert_tokenizer)
    
    print("\nBaseline Experiment Results:")
    print(f"Accuracy: {baseline_accuracy2}")
    print("Classification Report:")
    print(baseline_report2)

    print("\nBaseline Experiments completed.")


Running baseline experiment...

Extracting features...
Encoding labels...
Training and tuning model...
Best model parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Development set accuracy: 0.8582375478927203
Evaluating model...

Baseline Experiment Results:
Accuracy: 0.8417030567685589
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.80      0.73       205
           1       0.00      0.00      0.00        44
           2       0.90      0.91      0.91       667

    accuracy                           0.84       916
   macro avg       0.52      0.57      0.54       916
weighted avg       0.81      0.84      0.82       916


Baseline Experiments completed.


c:\Program Files\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


reviews, aspect_terms, sentiment_terms

In [4]:
def extract_features(reviews, aspect_terms, sentiment_terms, model, tokenizer):
    features = []
    for review, aspect_term, sentiment_term in zip(reviews, aspect_terms, sentiment_terms):
        # combine review, aspect term, and sentiment term into a single text
        combined_text = f"{review} [SEP] {aspect_term} [SEP] {sentiment_term}"
        
        inputs = tokenizer(combined_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        
        with torch.no_grad():
            outputs = model(**inputs)
        features.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(features)


def baseline_experiment(train_data, dev_data, test_data, model, tokenizer):
    
    print("\nExtracting features...")
    X_train = extract_features(train_data['review'], train_data['aspect_term'], train_data['sentiment_term'], model, tokenizer)
    X_dev = extract_features(dev_data['review'], dev_data['aspect_term'], dev_data['sentiment_term'], model, tokenizer)
    X_test = extract_features(test_data['review'], test_data['aspect_term'], test_data['sentiment_term'], model, tokenizer)
    
    print("Encoding labels...")
    le = LabelEncoder()
    le.fit(train_data['sentiment'].tolist() + dev_data['sentiment'].tolist() + test_data['sentiment'].tolist())
    y_train = le.transform(train_data['sentiment'])
    y_dev = le.transform(dev_data['sentiment'])
    y_test = le.transform(test_data['sentiment'])
    
    print("Training and tuning model...")
    best_classifier = train_and_tune_model(X_train, y_train, X_dev, y_dev)
    
    print("Evaluating model...")
    accuracy, report = evaluate_model(best_classifier, X_test, y_test)
    
    return best_classifier, le, accuracy, report

if __name__ == "__main__":
    bert_model = BertModel.from_pretrained(r'D:\AIProject\Bert\model')
    bert_tokenizer = BertTokenizer.from_pretrained(r'D:\AIProject\Bert\tokenizer')

    # restuarant
    rest_train_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_train.tsv')
    rest_test_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_test.tsv')
    rest_dev_data = load_tsv_data(r'D:\AIProject\data\restaurant\rest16_quad_dev.tsv')
    
    print("\nRunning baseline experiment...")
    baseline_classifier, baseline_le, baseline_accuracy, baseline_report = baseline_experiment(rest_train_data, rest_dev_data, rest_test_data, bert_model, bert_tokenizer)
    
    print("\nBaseline Experiment Results:")
    print(f"Accuracy: {baseline_accuracy}")
    print("Classification Report:")
    print(baseline_report)

    print("\nBaseline Experiments completed.")


Running baseline experiment...

Extracting features...
Encoding labels...
Training and tuning model...
Best model parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Development set accuracy: 0.8582375478927203
Evaluating model...

Baseline Experiment Results:
Accuracy: 0.8482532751091703
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.83      0.74       205
           1       0.56      0.11      0.19        44
           2       0.92      0.90      0.91       667

    accuracy                           0.85       916
   macro avg       0.71      0.62      0.61       916
weighted avg       0.85      0.85      0.84       916


Baseline Experiments completed.
